In [1]:
# load CIFAR-100 dataset
from torchvision import datasets
import torchvision.models as models
from torchvision.transforms import transforms
# import neccessary libraries
import os
import sys
import json
import requests
from tqdm import tqdm
import time
import datetime
import logging
import logging.handlers
import torch
import pandas as pd
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import os
from datasets import load_dataset
import torch.nn.parallel
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Subset
from torch.utils.data import Dataset
from torchvision import datasets, transforms
from torch.utils.data import Dataset
os.environ['CUDA_VISIBLE_DEVICES'] = '4'

/home/mingxu/miniconda3/envs/ML/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
transform_ori = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # 标准化
])

class CustomCIFAR100(Dataset):
    def __init__(self, root, train, download, transform=None):
        # 原始数据集
        self.original_dataset = datasets.CIFAR100(root=root, train=train, download=download, transform=transform_ori)
        # 只在增强数据集上应用变换
        self.augmented_dataset = datasets.CIFAR100(root=root, train=train, download=False, transform=transform)

    def __len__(self):
        # 数据集大小翻倍
        return 2 * len(self.original_dataset)

    def __getitem__(self, idx):
        if idx % 2 == 0:  # 偶数索引返回增强数据
            return self.augmented_dataset[idx // 2]
        else:  # 奇数索引返回原始数据
            return self.original_dataset[idx // 2]

# 定义增强变换
augmentations = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.RandomHorizontalFlip(p=1),
    transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.8, 1.2), shear=10),
    transforms.RandomRotation(10),
    transforms.RandomCrop(32, padding=4),
    transforms.Compose([transforms.RandomPerspective(distortion_scale=0.5)]),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])



# 加载数据集
train_dataset = CustomCIFAR100(root='./data', train=True, download=True, transform=augmentations)
test_dataset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_ori)

# 首先获取类别信息
classes = train_dataset.original_dataset.classes
print(classes)

# 初始化类计数器
class_counts = {i: 0 for i in range(100)}

# 遍历原始数据集的目标标签，每找到一个标签，计数加2（原始和增强）
for label in train_dataset.original_dataset.targets:
    class_counts[label] += 2  # 因为我们添加了原始和对应的增强图像

# 打印每个类别的样本数
for i in range(100):
    print(f"Class {i} has {classes[i]}, count: {class_counts[i]} samples")


Files already downloaded and verified
Files already downloaded and verified
['apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock', 'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur', 'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion', 'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse', 'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear', 'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine', 'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose', 'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake', 'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table', 'tank', 'telephone', 'television', 'tiger', 'tracto

In [3]:
# batch_size = 128 load dataset
from torch.utils.data import DataLoader

train_loader = DataLoader(dataset=train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=128, shuffle=False)


In [4]:
# print the train and test dataset sizes
print('Train: ', len(train_loader.dataset))
print('Test: ', len(test_loader.dataset))
# print the train and test batch sizes
print('Total Batch: ', len(train_loader))
print('Total Batch: ', len(test_loader))

Train:  100000
Test:  10000
Total Batch:  782
Total Batch:  79


In [5]:
import wandb
import random
wandb.init(
    project="DL_Classification_CIFAR-100",
    config={
    "learning_rate": 5e-3,
    "architecture": "Res2Net_enhance",
    "dataset": "CIFAR-100",
    "epochs": 50,
    }
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mingxu_zhang (mingxus-team). Use `wandb login --relogin` to force relogin


In [6]:
class Res2NetBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, scale=4):
        super(Res2NetBlock, self).__init__()
        self.scale = scale
        self.width = out_channels // scale
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.convs = nn.ModuleList([
            nn.Conv2d(self.width, self.width, kernel_size=3, stride=1, padding=1, bias=False) for _ in range(scale - 1)
        ])
        self.bns = nn.ModuleList([
            nn.BatchNorm2d(self.width) for _ in range(scale - 1)
        ])
        self.conv3 = nn.Conv2d(out_channels, out_channels, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        


        spx = torch.split(out, self.width, 1)
        spx = list(spx)
        out = self.convs[0](spx[0])
        out = self.relu(self.bns[0](out))
        

        for i in range(1, self.scale - 1):
            sp = self.convs[i](spx[i])
            sp = self.relu(self.bns[i](sp))
            out = torch.cat((out, sp), 1)
 

        if self.scale != 1:
            out = torch.cat((out, spx[-1]), 1)  # Adding the residual part of split not processed


        out = self.conv3(out)
        out = self.bn3(out)
        
        if self.shortcut is not None:
            residual = self.shortcut(x)

        out += residual
        out = self.relu(out)


        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=100, dropout_rate=0.5, scale=4):
        super(ResNet, self).__init__()
        self.in_channels = 32
        self.conv = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn = nn.BatchNorm2d(32)
        self.layer1 = self.make_layer(block, 32, num_blocks[0], stride=1, scale=scale)
        self.layer2 = self.make_layer(block, 64, num_blocks[1], stride=2, scale=scale)
        self.layer3 = self.make_layer(block, 128, num_blocks[2], stride=2, scale=scale)
        self.layer4 = self.make_layer(block, 256, num_blocks[3], stride=2, scale=scale)
        self.fc = nn.Linear(256, num_classes)

    def make_layer(self, block, out_channels, num_blocks, stride, scale):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride, scale))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn(self.conv(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out
        
    def evaluate(self, test_loader, criterion, use_cuda, index):
        # calculate the accuracy on the test set
        model.eval()
        test_loss = 0.0
        class_correct = list(0. for i in range(100))
        class_total = list(0. for i in range(100))
        for data, target in tqdm(test_loader):
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            output = model(data)
            loss = criterion(output, target)
            test_loss += loss.item()*data.size(0)
            _, pred = torch.max(output, 1)
            correct_tensor = pred.eq(target.data.view_as(pred))
            correct = np.squeeze(correct_tensor.numpy()) if not use_cuda else np.squeeze(correct_tensor.cpu().numpy())
            for i in range(len(target.data)):
                label = target.data[i]
                class_correct[label] += correct[i].item()
                class_total[label] += 1
        test_loss = test_loss/len(test_loader.dataset)
        for i in range(100):
            if class_total[i] > 0:
                # log accuracy of each class
                # wandb.log({"acc_{}".format(classes[i]): class_correct[i] / class_total[i]})
                wandb.log({"acc_model_{}".format(index): class_correct[i] / class_total[i]})
                #print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (str(i), 100 * class_correct[i] / class_total[i], np.sum(class_correct[i]), np.sum(class_total[i])))
            else:
                print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))  
        #wandb log the average acc
        # wandb.log({"acc": np.sum(class_correct) / np.sum(class_total)})
        # print average acc
        print('Test Accuracy (Overall): %2d%% (%2d/%2d)' % (100. * np.sum(class_correct) / np.sum(class_total), np.sum(class_correct), np.sum(class_total)))
        

    def train_model(model, train_loader, valid_loader, epochs, optimizer, criterion, use_cuda, save_path):
        valid_loss_min = np.Inf
        count = 0
        for epoch in range(1, epochs+1):
            train_loss = 0.0
            valid_loss = 0.0
            model.train()
            for data, target in tqdm(train_loader):
                if use_cuda:
                    data, target = data.cuda(), target.cuda()
                optimizer.zero_grad()
                output = model(data)
                loss = criterion(output, target)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()*data.size(0)
            model.eval()
            for data, target in valid_loader:
                if use_cuda:
                    data, target = data.cuda(), target.cuda()
                output = model(data)
                loss = criterion(output, target)
                valid_loss += loss.item()*data.size(0)
            train_loss = train_loss/len(train_loader.sampler)
            valid_loss = valid_loss/len(valid_loader.sampler)
            model.evaluate( valid_loader, criterion, use_cuda)
            wandb.log({"training_loss": train_loss, "val_loss": valid_loss})  

            print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch, train_loss, valid_loss))
            if valid_loss <= valid_loss_min:
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min, valid_loss))
                torch.save(model.state_dict(), save_path)
                valid_loss_min = valid_loss
                count = 0
            else :
                count = count + 1
                if count > 10:
                    break
            torch.save(model.state_dict(), save_path + str(epoch%5))

    def predict(model, test_loader, use_cuda):
        model.eval()  # Ensure the model is in evaluation mode
        predictions = []
        with torch.no_grad():  # Turn off gradients for validation, saves memory and computations
            for data, _ in tqdm(test_loader):
                if use_cuda:
                    data = data.cuda()  # Move data to GPU if CUDA is enabled
                output = model(data)
                _, pred = torch.max(output, 1)  # Get the index of the max log-probability
                pred = pred.cpu().numpy() if use_cuda else pred.numpy()  # Move data to CPU if CUDA is used
                predictions.extend(pred)
        return predictions

    def predict_poss(model, test_loader, use_cuda):
        model.eval()  # Ensure the model is in evaluation mode
        probabilities = []
        with torch.no_grad():  # Turn off gradients for validation, saves memory and computations
            for data, _ in tqdm(test_loader):
                if use_cuda:
                    data = data.cuda()  # Move data to GPU if CUDA is enabled

                output = model(data)
                prob = F.softmax(output, dim=1)  # Compute the probability distribution over classes

                if use_cuda:
                    prob = prob.cpu()  # Move data to CPU if CUDA is used

                probabilities.append(prob.numpy())  # Store the probabilities

        # 使用 np.vstack 以确保即使批量大小不同也能正确合并
        return np.vstack(probabilities)        

In [7]:
num_blocks = [5, 6, 7, 8]
model = ResNet(Res2NetBlock, num_blocks, num_classes=100, dropout_rate=0, scale=4)

print(model)

ResNet(
  (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): Res2NetBlock(
      (conv1): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (convs): ModuleList(
        (0-2): 3 x Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (bns): ModuleList(
        (0-2): 3 x BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv3): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (shortcut): Sequential()
    )
    (1): Res2NetBlock(
      (conv1): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)

In [8]:
os.environ["CUDA_VISIBLE_DEVICES"] = "4"
criterion = nn.CrossEntropyLoss()
model0 = ResNet(Res2NetBlock, num_blocks, num_classes=100, dropout_rate=0, scale=4)
model1 = ResNet(Res2NetBlock, num_blocks, num_classes=100, dropout_rate=0, scale=4)
model2 = ResNet(Res2NetBlock, num_blocks, num_classes=100, dropout_rate=0, scale=4)
model3 = ResNet(Res2NetBlock, num_blocks, num_classes=100, dropout_rate=0, scale=4)
model4 = ResNet(Res2NetBlock, num_blocks, num_classes=100, dropout_rate=0, scale=4)
use_cuda = torch.cuda.is_available()
if use_cuda:
    model0 = model0.cuda()
    model1 = model1.cuda()
    model2 = model2.cuda()
    model3 = model3.cuda()
    model4 = model4.cuda()

# load model
model0.load_state_dict(torch.load('Res2Net_baseline_model_enhance0.pt'))
model1.load_state_dict(torch.load('Res2Net_baseline_model_enhance1.pt'))
model2.load_state_dict(torch.load('Res2Net_baseline_model_enhance2.pt'))
model3.load_state_dict(torch.load('Res2Net_baseline_model_enhance3.pt'))
model4.load_state_dict(torch.load('Res2Net_baseline_model_enhance4.pt'))

<All keys matched successfully>

In [9]:
'''
model = ResNet(Res2NetBlock, num_blocks, num_classes=100, dropout_rate=0, scale=4)
use_cuda = torch.cuda.is_available()
if use_cuda:
    model = model.cuda()
# train model
criterion = nn.CrossEntropyLoss()
model.load_state_dict(torch.load('Res2Net_baseline_model_enhance4.pt'))
model.evaluate(test_loader, criterion, use_cuda, 'Res2Net_baseline_enhance4')
'''

"\nmodel = ResNet(Res2NetBlock, num_blocks, num_classes=100, dropout_rate=0, scale=4)\nuse_cuda = torch.cuda.is_available()\nif use_cuda:\n    model = model.cuda()\n# train model\ncriterion = nn.CrossEntropyLoss()\nmodel.load_state_dict(torch.load('Res2Net_baseline_model_enhance4.pt'))\nmodel.evaluate(test_loader, criterion, use_cuda, 'Res2Net_baseline_enhance4')\n"

In [10]:
# 使用多数投票的方式进行集成
'''
model0.eval()
model1.eval()
model2.eval()
model3.eval()
model4.eval()

test_preds0 = torch.tensor(model0.predict(test_loader, use_cuda), dtype=torch.long)
test_preds1 = torch.tensor(model1.predict(test_loader, use_cuda), dtype=torch.long)
test_preds2 = torch.tensor(model2.predict(test_loader, use_cuda), dtype=torch.long)
test_preds3 = torch.tensor(model3.predict(test_loader, use_cuda), dtype=torch.long)
test_preds4 = torch.tensor(model4.predict(test_loader, use_cuda), dtype=torch.long)

stacked_preds = torch.stack([test_preds0, test_preds1, test_preds2, test_preds3, test_preds4], dim=0)

# Apply torch.mode to find the most common prediction along the stacked dimension
test_preds = torch.mode(stacked_preds, dim=0).values

# Ensure that the dataset has an attribute 'targets' or adjust according to your dataset structure
class_correct = list(0. for i in range(100))
class_total = list(0. for i in range(100))
for i in range(len(test_loader.dataset.targets)):
    label = test_loader.dataset.targets[i]
    class_correct[label] += int(test_preds[i].item() == label)

    class_total[label] += 1

# Log ensemble accuracies to wandb and print them
for i in range(100):
    if class_total[i] > 0:
        wandb.log({"res2net_acc_ensemble_voting5": class_correct[i] / class_total[i]})
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (str(i), 100 * class_correct[i] / class_total[i], class_correct[i], class_total[i]))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % i)

# Calculate and print overall accuracy
print('Test Accuracy (Overall): %2d%% (%2d/%2d)' % (100. * sum(class_correct) / sum(class_total), sum(class_correct), sum(class_total)))

'''


'\nmodel0.eval()\nmodel1.eval()\nmodel2.eval()\nmodel3.eval()\nmodel4.eval()\n\ntest_preds0 = torch.tensor(model0.predict(test_loader, use_cuda), dtype=torch.long)\ntest_preds1 = torch.tensor(model1.predict(test_loader, use_cuda), dtype=torch.long)\ntest_preds2 = torch.tensor(model2.predict(test_loader, use_cuda), dtype=torch.long)\ntest_preds3 = torch.tensor(model3.predict(test_loader, use_cuda), dtype=torch.long)\ntest_preds4 = torch.tensor(model4.predict(test_loader, use_cuda), dtype=torch.long)\n\nstacked_preds = torch.stack([test_preds0, test_preds1, test_preds2, test_preds3, test_preds4], dim=0)\n\n# Apply torch.mode to find the most common prediction along the stacked dimension\ntest_preds = torch.mode(stacked_preds, dim=0).values\n\n# Ensure that the dataset has an attribute \'targets\' or adjust according to your dataset structure\nclass_correct = list(0. for i in range(100))\nclass_total = list(0. for i in range(100))\nfor i in range(len(test_loader.dataset.targets)):\n    la

In [11]:
# 确保模型在评估模式

model0.eval()
model1.eval()
model2.eval()
model3.eval()
model4.eval()

# 获取每个模型的概率预测
prob_preds0 = np.array(model0.predict_poss(test_loader, use_cuda))
prob_preds1 = np.array(model1.predict_poss(test_loader, use_cuda))
prob_preds2 = np.array(model2.predict_poss(test_loader, use_cuda))
prob_preds3 = np.array(model3.predict_poss(test_loader, use_cuda))
prob_preds4 = np.array(model4.predict_poss(test_loader, use_cuda))

# 将所有模型的概率相加
summed_probs = prob_preds0 + prob_preds1 + prob_preds2 + prob_preds3 + prob_preds4

# 选择概率总和最大的类别作为最终预测
test_preds = np.argmax(summed_probs, axis=1)

# 确保数据集具有属性 'targets' 或根据您的数据集结构进行调整
class_correct = list(0. for i in range(100))
class_total = list(0. for i in range(100))
for i, label in enumerate(test_loader.dataset.targets):
    class_correct[label] += int(test_preds[i] == label)
    class_total[label] += 1

# 打印每个类别的精度
for i in range(100):
    if class_total[i] > 0:
        wandb.log({"Res2Net_acc_ensemble_possibility": class_correct[i] / class_total[i]})
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (str(i), 100 * class_correct[i] / class_total[i], class_correct[i], class_total[i]))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % i)

# 计算并打印总体精度
overall_accuracy = 100. * sum(class_correct) / sum(class_total)
print('Test Accuracy (Overall): %2d%% (%2d/%2d)' % (100. * sum(class_correct) / sum(class_total), sum(class_correct), sum(class_total)))


  0%|          | 0/79 [00:00<?, ?it/s]

100%|██████████| 79/79 [00:03<00:00, 25.78it/s]

Test Accuracy of     0: 85% (85/100)
Test Accuracy of     1: 80% (80/100)
Test Accuracy of     2: 56% (56/100)
Test Accuracy of     3: 48% (48/100)
Test Accuracy of     4: 54% (54/100)
Test Accuracy of     5: 58% (58/100)
Test Accuracy of     6: 77% (77/100)
Test Accuracy of     7: 72% (72/100)
Test Accuracy of     8: 85% (85/100)
Test Accuracy of     9: 80% (80/100)
Test Accuracy of    10: 52% (52/100)
Test Accuracy of    11: 45% (45/100)
Test Accuracy of    12: 75% (75/100)
Test Accuracy of    13: 54% (54/100)
Test Accuracy of    14: 53% (53/100)
Test Accuracy of    15: 71% (71/100)
Test Accuracy of    16: 69% (69/100)
Test Accuracy of    17: 82% (82/100)
Test Accuracy of    18: 63% (63/100)
Test Accuracy of    19: 53% (53/100)
Test Accuracy of    20: 80% (80/100)
Test Accuracy of    21: 89% (89/100)
Test Accuracy of    22: 58% (58/100)
Test Accuracy of    23: 84% (84/100)
Test Accuracy of    24: 80% (80/100)
Test Accuracy of    25: 56% (56/100)
Test Accuracy of    26: 57% (57/100)
T